In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import pickle

In [ ]:
import yaml
import logging
from datetime import datetime

# YAML config
try:
    with open(r".\config.yaml", "r") as f:
        config = yaml.safe_load(f)
except Exception as e:
    raise

# Logger
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(name)s - %(funcName)s - %(message)s",
    filename=config["log_dir"] +
    f"{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.log",
    filemode="w"
)
logger = logging.getLogger(__name__)

logger.info("Config file and logger setup completed.")

In [ ]:
# Load the dataset
try:
    df = pd.read_csv(config["data_path"])
    logger.info("Dataset loaded successfully.")
except FileNotFoundError:
    logger.error("Error: CSV file not found.  Please check the file path.")

In [ ]:
# Data Exploration
df = df.set_index("car_ID")

logger.info("Initial DataFrame shape: {}".format(df.shape))

In [ ]:
# Handle Missing Values
if df.isnull().sum().sum() > 0:
    print("Missing values found. Imputation will be performed.")
    logger.warning("Missing values found. Imputation will be performed.")
else:
    print("No missing values found.")
    logger.info("No missing values found.")

In [ ]:
# Data Inspection
df.info()
print(df.describe().T)

In [ ]:
# Feature Engineering
df["brand"] = df["CarName"].str.split(" ", n=1, expand=True)[0]
df["model"] = df["CarName"].str.split(" ", n=1, expand=True)[
    1].str.replace(" ", "")

df = df.drop("CarName", axis=1)

In [ ]:
# Declaring columns
numeric_cols = [col for col in df.columns if pd.api.types.is_numeric_dtype(
    df[col]) and col != "price"]
categorical_cols = [
    col for col in df.columns if col not in numeric_cols and col != "price"]

In [ ]:
df = df[(np.abs(stats.zscore(df[numeric_cols])) < 3).all(axis=1)]

In [ ]:
# One-Hot Encoding
logger.info("Applying One-Hot Encoding...")
encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
preprocessor = ColumnTransformer(
    [("onehot", encoder, categorical_cols)], remainder="passthrough")
df_processed = pd.DataFrame(preprocessor.fit_transform(
    df), columns=preprocessor.get_feature_names_out())

In [ ]:
numeric_cols = [column for column in list(
    df_processed.columns) if not column.startswith("onehot")]

In [ ]:
# Scaling Numeric Features
scaler = StandardScaler()
df_processed[numeric_cols] = scaler.fit_transform(df_processed[numeric_cols])

In [ ]:
# Split Data
X = df_processed
y = df["price"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [ ]:
# Model Training
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

In [ ]:
# Model Evaluation
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"R2 Score: {r2:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"Mean Squared Error: {mse:.4f}")

In [ ]:
# Save the Model
pickle.dump(model, open(config["save_model_name"], "wb"))
logger.info(f"Model saved to {config["save_model_name"]}")